In [9]:
# pip install opencv-python
import cv2

In [11]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

In [17]:
# Read the image where you want to detect faces
image = cv2.imread('image.jpg')


In [21]:
# Convert image to grayscale (face detection works better on grayscale images)
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

In [23]:
# Detect faces in the image
faces = face_cascade.detectMultiScale(gray_image, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

In [25]:
# Detect faces in the image
faces = face_cascade.detectMultiScale(gray_image, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

In [27]:
# Draw rectangles around the faces
for (x, y, w, h) in faces:
    cv2.rectangle(image, (x, y), (x+w, y+h), (255, 0, 0), 2)

In [29]:
# Display the output image with detected faces
cv2.imshow('Faces Detected', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [37]:
import cv2

# Load the Haar cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Start video capture
cap = cv2.VideoCapture(0)

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the image
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Draw rectangles around detected faces
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)

    # Display the resulting frame
    cv2.imshow('Face Detection', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture and close windows
cap.release()
cv2.destroyAllWindows()

In [43]:
import cv2
import time
import winsound  # for playing alarm sound on Windows, you can use other libraries for other platforms

# Load Haar cascades for face and eye detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

# Start video capture
cap = cv2.VideoCapture(0)

distraction_start_time = None  # to track distraction duration

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Convert to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect face
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    distraction_detected = False  # Track if the driver is distracted

    for (x, y, w, h) in faces:
        # Draw rectangle around face
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

        # Region of interest for the eyes
        roi_gray = gray[y:y + h, x:x + w]
        roi_color = frame[y:y + h, x:x + w]

        # Detect eyes
        eyes = eye_cascade.detectMultiScale(roi_gray)
        if len(eyes) < 2:  # Assume driver is not looking straight ahead
            distraction_detected = True
            if distraction_start_time is None:
                distraction_start_time = time.time()  # Start distraction timer
        else:
            distraction_start_time = None  # Reset timer if driver is attentive

    # If distraction is detected for more than 3 seconds, trigger alarm
    if distraction_detected:
        if distraction_start_time is not None and time.time() - distraction_start_time > 3:
            cv2.putText(frame, "DISTRACTION DETECTED!", (100, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            winsound.Beep(2000, 500)  # Play alarm sound (adjust for your system)
    else:
        distraction_start_time = None

    # Display the frame
    cv2.imshow('Driver Attention System', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture and close windows
cap.release()
cv2.destroyAllWindows()

In [1]:
import cv2
import mediapipe as mp
import pywhatkit as kit

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()
mp_draw = mp.solutions.drawing_utils

# Initialize webcam
cap = cv2.VideoCapture(0)

# Music player control logic
def control_music(gesture):
    if gesture == "one_finger":
        print("Play/Pause music")
        kit.playonyt("Shape of You")  # Example of playing music on YouTube
    elif gesture == "two_fingers":
        print("Next track")
    elif gesture == "palm_open":
        print("Stop music")

while True:
    success, img = cap.read()
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Process the frame with MediaPipe hands
    result = hands.process(img_rgb)
    
    if result.multi_hand_landmarks:
        for hand_lms in result.multi_hand_landmarks:
            mp_draw.draw_landmarks(img, hand_lms, mp_hands.HAND_CONNECTIONS)
            
            # Extract landmark coordinates for specific gesture detection
            landmarks = hand_lms.landmark
            
            # Example: Gesture recognition using landmarks
            # Detecting one finger gesture (finger #8 extended)
            finger_tip = landmarks[8].y
            finger_base = landmarks[6].y
            
            if finger_tip < finger_base:
                control_music("one_finger")  # Play/Pause

            # Detecting two fingers (fingers #8 and #12 extended)
            finger_tip_2 = landmarks[12].y
            finger_base_2 = landmarks[10].y
            
            if finger_tip < finger_base and finger_tip_2 < finger_base_2:
                control_music("two_fingers")  # Next track

            # Detecting palm open (all fingers extended)
            if all(landmarks[i].y < landmarks[i - 2].y for i in [8, 12, 16, 20]):
                control_music("palm_open")  # Stop music
    
    # Display the result
    cv2.imshow("Hand Gesture Control", img)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

D:\anaconda\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Play/Pause music
Play/Pause music
Play/Pause music
Play/Pause music
Next track
Stop music
Play/Pause music
Next track
Stop music
Play/Pause music
Next track
Stop music
Play/Pause music
Play/Pause music
Play/Pause music
Play/Pause music
Next track
Play/Pause music
Next track
Play/Pause music
Play/Pause music
Play/Pause music
Next track
Stop music
Play/Pause music
Next track
Stop music
Play/Pause music
Next track
Stop music
Play/Pause music
Next track
Stop music
Play/Pause music
Next track
Stop music
Play/Pause music
Next track
Stop music
Play/Pause music
Next track
Stop music
Play/Pause music
Next track
Stop music
Play/Pause music
Next track
Stop music
Play/Pause music
Next track
Stop music
Play/Pause music
Next track
Stop music
Play/Pause music
Play/Pause music
Play/Pause music
Next track
Play/Pause music
Next track
Play/Pause music
Next track
Play/Pause music


In [9]:
pip install mediapipe --user

  Using cached mediapipe-0.10.14-cp312-cp312-win_amd64.whl.metadata (9.9 kB)
  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached jax-0.4.34-py3-none-any.whl.metadata (22 kB)
  Using cached jaxlib-0.4.34-cp312-cp312-win_amd64.whl.metadata (1.0 kB)
  Using cached opencv_contrib_python-4.10.0.84-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Using cached sounddevice-0.5.1-py3-none-win_amd64.whl.metadata (1.4 kB)
  Using cached ml_dtypes-0.5.0-cp312-cp312-win_amd64.whl.metadata (22 kB)
Using cached mediapipe-0.10.14-cp312-cp312-win_amd64.whl (50.8 MB)
Using cached sounddevice-0.5.1-py3-none-win_amd64.whl (363 kB)
Using cached absl_py-2.1.0-py3-none-any.whl (133 kB)
Using cached jax-0.4.34-py3-none-any.whl (2.1 MB)
Using cached jaxlib-0.4.34-cp312-cp312-win_amd64.whl (55.3 MB)
Using cached opencv_contrib_python-4.10.0.84-cp37-abi3-win_amd64.whl (45.5 MB)
Using cached ml_dtypes-0.5.0-cp312-cp312-win_amd64.whl (213 kB)
Note: you may need to restart the kernel to use upd

In [8]:

import cv2
import mediapipe as mp
import numpy as np
import time
import winsound

# Load face detection using OpenCV
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Initialize MediaPipe Hands for finger gesture detection
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()
mp_draw = mp.solutions.drawing_utils

# Driver fatigue and distraction detection variables
last_looked_at_road_time = time.time()
fatigue_detected = False
distraction_detected = False

# Start video capture
cap = cv2.VideoCapture(0)

# Function to play alarm
def play_alarm():
    winsound.Beep(1000, 1000)  # 1kHz beep for 1 second

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # Convert to grayscale for face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Detect face
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    
    # Check if face is detected
    if len(faces) > 0:
        # Draw rectangle around the face
        (x, y, w, h) = faces[0]
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        
        # Detect face landmarks to track eye openness (for drowsiness detection)
        face_roi = frame[y:y + h, x:x + w]
        gray_roi = gray[y:y + h, x:x + w]
        
        # Use MediaPipe to detect hand gestures (if available)
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(rgb_frame)
        
        # Process detected hand gestures
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                
                # Count the number of fingers raised (simple gesture recognition)
                fingers_raised = 0
                for i in [4, 8, 12, 16, 20]:  # Check landmarks for fingers
                    if hand_landmarks.landmark[i].y < hand_landmarks.landmark[i - 2].y:
                        fingers_raised += 1
                
                # If thumbs-up (1 finger raised), consider the driver alert
                if fingers_raised == 1:
                    cv2.putText(frame, "Driver is alert (Thumbs Up)", (20, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                    last_looked_at_road_time = time.time()  # Reset the time when alert gesture is detected
                else:
                    # If no alert gesture, detect distraction
                    if time.time() - last_looked_at_road_time > 5:  # 5 seconds without face in front
                        distraction_detected = True
                        cv2.putText(frame, "Distraction Detected!", (20, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                        play_alarm()
                    
    else:
        # If no face detected, assume the driver is distracted
        distraction_detected = True
        cv2.putText(frame, "No Face Detected! Please pay attention!", (20, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        play_alarm()
    
    # Show the frame
    cv2.imshow('Driver Fatigue & Distraction Detection', frame)
    
    # Exit the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [14]:
import cv2
import mediapipe as mp
import numpy as np

# Initialize MediaPipe Hands for hand tracking
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7)
mp_draw = mp.solutions.drawing_utils

# For drawing on the screen
drawing_color = (0, 255, 0)  # Green color for drawing

# Variables to store gestures
gestures = []
current_expression = ""

# Define function to recognize numbers from hand gestures
def recognize_gesture(landmarks):
    # You can create a simple number gesture recognition based on hand landmarks.
    # For now, just return a sample gesture for demonstration.
    if landmarks[8].x > landmarks[4].x and landmarks[12].x > landmarks[4].x:
        return "1"
    if landmarks[8].x < landmarks[4].x and landmarks[12].x > landmarks[4].x:
        return "2"
    if landmarks[8].x > landmarks[4].x and landmarks[12].x < landmarks[4].x:
        return "3"
    return ""

# Initialize webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Flip the frame for a mirror view
    frame = cv2.flip(frame, 1)

    # Convert the frame to RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame with MediaPipe hands
    results = hands.process(rgb_frame)
    
    # Get hand landmarks
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # Draw landmarks and connections
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # Recognize the number or symbol being drawn
            gesture = recognize_gesture(hand_landmarks.landmark)

            if gesture:
                gestures.append(gesture)

    # Display the recognized expression
    if len(gestures) > 0:
        current_expression = "".join(gestures)

    # Display the current expression on the frame
    cv2.putText(frame, f"Expression: {current_expression}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

    # Evaluate the expression when Enter key is pressed
    if cv2.waitKey(1) & 0xFF == ord('\r'):  # Enter key
        try:
            result = eval(current_expression)
            cv2.putText(frame, f"Result: {result}", (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        except Exception as e:
            cv2.putText(frame, "Error!", (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # Display the final frame
    cv2.imshow("Gesture Calculator", frame)

    # Exit the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

D:\anaconda\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
